# Power price validation
Routines for price validation comprising
* Comparison of price statistics
* Comparison of price patterns
* Comparison of negative price and scarcity price events
* Calculation of error metrics (MAE, RSME)

## Package imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pommesevaluation.price_validation import (
    read_and_reshape_historical_prices, compare_negative_price_distribution,
    draw_price_plot, draw_weekly_plot, draw_price_duration_plot, calculate_error_metrics
)

## Set parameters
Specify path and filenames where data is stored:
* Power price results from the model.
* Historical day-ahead prices (used for validation)

In [ ]:
simulation_year = 2017
path_results = "./model_results/"
path_plots = "./plots/"
path_historical_prices = "./data/prices/"

model_file_name = f"dispatch_LP_start-{simulation_year}-01-01_364-days_simple_complete_power-prices.csv"

## Read in data
* Historical prices for years from 2017 to 2021 (simulate year used for validation)
* Model prices for the respective simulation year

In [ ]:
historical_prices_fnames = {
    2017: f"{path_historical_prices}auction_spot_prices_germany_austria_2017.csv",
    2018: [
        f"{path_historical_prices}auction_spot_prices_germany_austria_2018.csv", 
        f"{path_historical_prices}auction_spot_prices_germany_luxembourg_2018.csv"
    ]
}

for year in range(2019, 2022):
    historical_prices_fnames[year] = f"{path_historical_prices}auction_spot_prices_germany_luxembourg_{year}.csv"
    
historical_prices = dict()

for year, file_name in historical_prices_fnames.items():
    historical_prices[year] = read_and_reshape_historical_prices(year, file_name)

In [ ]:
model_prices = pd.read_csv(
    path_results + model_file_name, sep=",", decimal=".", index_col = 0, infer_datetime_format=True
)
model_prices.index = pd.to_datetime(model_prices.index)
model_prices = model_prices.rename(columns={"Power price":"model_price"})

## Visualize historical prices and calculate some statistics
* Plot power prices for years 2017 to 2021
* Calculate summary statistics for all years

In [ ]:
loc_map = {
    2017: (0, 0),
    2018: (0, 1),
    2019: (0, 2),
    2020: (1, 0),
    2021: (1, 1),
}

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20, 10), sharey=True)

for key, val in historical_prices.items():
    val.plot(ax=axes[loc_map[key]])
    axes[loc_map[key]].set_title(key)

fig.delaxes(axes[1, 2])
axes[0, 0].set_ylabel("power price in €/MWh")
axes[1, 0].set_ylabel("power price in €/MWh")
plt.tight_layout()
plt.show()

In [ ]:
stats_df = pd.DataFrame(index=historical_prices[2017].describe().index, columns=range(2017, 2022))
stats_df.loc["negative_hours"] = 0
stats_df.loc["scarcity_hours"] = 0

for key, val in historical_prices.items():
    stats_df[key] = val.describe()
    stats_df.at["negative_hours", key] = val.loc[val["historical_price"] < 0].count()
    stats_df.at["scarcity_hours", key] = val.loc[val["historical_price"] == 3000].count()
    
stats_df

# Validate model results with historical prices
Model results for a historical year are evaluated against historical day-ahead prices.
* Summary statistics are calculated for comparison.
* Negative price distributions are compared against each other.
* The annual price time series and duration curves are plotted against each other.
* Weekly price time series are written to disk.
* Mean average error (note: for evaluating estimates for the median) and root mean squared error (note: for evaluating estimates for the mean value) are evaluated.

## Compare summary statistics

In [ ]:
stats_comparison_df = pd.DataFrame(index=stats_df.index, columns=["historical", "model"])

stats_comparison_df["historical"] = historical_prices[simulation_year].describe()
stats_comparison_df["model"] = model_prices.describe()
stats_comparison_df.at["negative_hours", "historical"] = historical_prices[simulation_year].loc[historical_prices[simulation_year].historical_price < 0].count()
stats_comparison_df.at["negative_hours", "model"] = model_prices.loc[model_prices.model_price < 0].count()
stats_comparison_df.at["scarcity_hours", "historical"] = historical_prices[simulation_year].loc[historical_prices[simulation_year].historical_price == 3000].count()
stats_comparison_df.at["scarcity_hours", "model"] = model_prices.loc[model_prices.model_price == 3000].count()

stats_comparison_df

## Compare negative price distribution

In [ ]:
compare_negative_price_distribution(model_prices, historical_prices, simulation_year)

## Compare annual and weekly prices against eacht other

In [ ]:
if simulation_year < 2022:
    # Concatenate model results and historical ones
    prices = pd.concat([historical_prices[simulation_year], model_prices], axis=1)

    draw_price_plot(
        power_prices=prices, 
        color=["b", "r"],
        title=f"Power price time series comparison for {simulation_year}",
        y_min_max=True,
        show=True,
        save=True,
        file_name=f"power_price_time_series_{simulation_year}"
    )

In [ ]:
if simulation_year < 2022:
    draw_weekly_plot(prices, simulation_year)

In [ ]:
if simulation_year < 2022:
    draw_price_duration_plot(
        model_prices, historical_prices[simulation_year], show=True, save=True, file_name=f"power_price_duration_curve_{simulation_year}"
    )

## Calculate error values
Calculate mean absolute and root mean squared error for model to get an impression on model results quality

In [ ]:
if simulation_year < 2022:
    error_metrics = calculate_error_metrics(historical_prices[simulation_year], model_prices)
    print("Mean absolute error for model results: {:.2f} \nRoot mean squared error for model results: {:.2f}".format(error_metrics["MAE"], error_metrics["RMSE"]))

# Check model results in isolation (simulation of future years)
For future years, no comparison againts historical values is possible. Thus, model results can only be considered in isolation.

### Calculate summary statistics

In [ ]:
stats_comparison_df = pd.DataFrame(index=stats_df.index, columns=["model"])

stats_comparison_df["model"] = model_prices.describe()
stats_comparison_df.at["negative_hours", "model"] = model_prices.loc[model_prices.model_price < 0].count()
stats_comparison_df.at["scarcity_hours", "model"] = model_prices.loc[model_prices.model_price == 3000].count()

stats_comparison_df

### Plot price pattern

In [ ]:
draw_price_plot(
    model_prices,
    color="b",
    title=f"Power price time series comparison for {simulation_year}",
    y_min_max=False,
    show=True,
    save=True,
    file_name=f"power_price_time_series_{simulation_year}"
)

In [ ]:
model_prices.idxmin()

### Plot price duration curve

In [ ]:
model_prices_sorted = model_prices.sort_values(
        by="model_price", ascending=False
    ).reset_index(drop=True)

draw_price_plot(
    model_prices_sorted,
    color="b",
    title=f"Price duration curve for {simulation_year}",
    y_min_max=False,
    show=True,
    save=True,
    file_name=f"power_price_time_series_{simulation_year}"
)